In [1]:
import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import talib

from __future__ import division      #除数可以显示为float

import time                 #使用time stamp
import datetime             


import matplotlib.pyplot as plt


stock_to_choose = get_fundamentals(query(
        valuation.code, valuation.pe_ratio, 
        valuation.pb_ratio,valuation.market_cap, 
        indicator.eps, indicator.inc_net_profit_annual
    ).filter(
#         indicator.inc_net_profit_annual > 50
#         valuation.pe_ratio < 100,
        valuation.pe_ratio > 2
    ).order_by(
        valuation.market_cap.desc()
    ), date=None)

stock_list_all = list(stock_to_choose['code'])

# print stock_list_all

选取股票

In [2]:
stock_list_all = stock_list_all[:]
# stock_list_all = ['002558.XSHE', '600362.XSHG', '600516.XSHG', '603260.XSHG', '000898.XSHE', '002202.XSHE', '600271.XSHG', '600066.XSHG', '600377.XSHG', '002602.XSHE']
# stock_list_all = ['000651.XSHE,000858.XSHE']
# print stock_list_all

In [3]:
#统计汇总
stock_stat = {}          #{股票：{日期：胜败结果}}

 
for stock_name in set(stock_list_all):
    stat = {}
    
    stock_price = get_price(stock_name, count = 2000, end_date='2018-5-1', frequency='6d', 
                            fields=['open', 'close'], fq='none')

 
    stock_price[stock_price['close']==0]=np.nan
    close_price= stock_price.dropna()

    close_price = stock_price['close']
    
    if len(close_price) > 50:

        dif, dea, macd = talib.MACD(stock_price['close'].values, fastperiod=13, slowperiod=26, signalperiod=9)
        
        id_x_gold = np.where((macd[:-1] < 0) & (macd[1:] > 0))[0] + 1

        id_x_gold = [elem for elem in id_x_gold if elem < (len(close_price) - 10) ]  # 防止越界
               
        
        for gold_date in id_x_gold:                  #逐个判断金叉的时刻
            
            if dif[gold_date] > 0 :                             #dif>0，表明处于多头市场

                
                stat[close_price.index[gold_date]] = "error"
                
                # 开始赋值判断
                days = 0
                    
                for price_cmp in close_price[gold_date:gold_date + 10]:
                    if price_cmp < 0.985*close_price[gold_date]:
                        stat[close_price.index[gold_date]] = "lose"
                        break
                        
                    if price_cmp > 1.02*close_price[gold_date]:
                        stat[close_price.index[gold_date]] = "win"
                        break

                    if days > 4 :
                        stat[close_price.index[gold_date]] = "even"
                        break
                        
                    days += 1
    
#     print stat
    stock_stat[stock_name] = stat
#     stock_stat[stock_name] = {}
#     for date in attempt_count.keys():
#         stock_stat[stock_name][date] =  stat[date] if stat.has_key(date) else 0
    
# print stock_stat

统计正确次数、失败次数


In [4]:
# print stock_stat

win_count = 0
lose_count = 0
even_count = 0

for stock_name in stock_stat.keys():
    for date in stock_stat[stock_name].keys():
        if stock_stat[stock_name][date] == "win":
            win_count += 1
        if stock_stat[stock_name][date] == "lose":
            lose_count += 1
        if stock_stat[stock_name][date] == "even":
            even_count += 1
            
print "正确次数：%d " % win_count
print "失败次数：%d " % lose_count
print "平局次数：%d " % even_count
print "胜率：%f " % (float(win_count+even_count)/float(win_count+lose_count+even_count))

正确次数：5870 
失败次数：5488 
平局次数：143 
胜率：0.522824 


按年度进行分组，看每一年胜者组下一年仍然是胜者的概率
确定分组是否对胜率有帮助

In [5]:
# print stock_stat

winner_list = {}           #{year:[stock1,stock2]}  是单纯的股票列表，里面存在重复的股票，年度键值是str
loser_list = {}
evener_list = {}

for stock_name in stock_stat.keys():
    
    for date in stock_stat[stock_name].keys():
        
        if stock_stat[stock_name][date] == 'win':
            if winner_list.has_key(str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)):
                winner_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)].append(stock_name)
            else:
                winner_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)] = [stock_name]
            

        elif stock_stat[stock_name][date] == 'lose':
            if loser_list.has_key(str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)):
                loser_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)].append(stock_name)
            else:
                loser_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)] = [stock_name]
            
            
        elif stock_stat[stock_name][date] == 'even':
            if evener_list.has_key(str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)):
                evener_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)].append(stock_name)
            else:
                evener_list[str(datetime.datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S").year)] = [stock_name]
                
# print winner_list
# print loser_list
# print evener_list

开始逐年计算胜率
按照每年统计胜者的列表，看下一年是否胜者是否仍然存在

In [6]:
#  winner_list:{year:[stock1,stock2]}  是单纯的股票列表，里面存在重复的股票，年度键值是str

winner_stat = {}         #{年度：stock}   代表stock胜率高于50%的股票，唯一，年度键值是int
loser_stat = {}
even_stat = {}

# 以年度为单位来统计
for yearth in winner_list.keys():
    winner_stat[int(yearth)] = []         
    loser_stat[int(yearth)] = []
    even_stat[int(yearth)] = []
    
    
    # stock_list_all 是之前的股票列表
    for stock_name in stock_list_all: 
        wins = 0
        loses = 0
        evens = 0

        wins = sum(map(lambda x: x == stock_name, winner_list[str(yearth)]))
        loses = sum(map(lambda x: x == stock_name, loser_list[str(yearth)]))
        evens = sum(map(lambda x: x == stock_name, winner_list[str(yearth)]))
          

        
        if (wins + evens)/(wins + loses + evens) > 0.6: 
            winner_stat[int(yearth)].append(stock_name)
        elif (wins + evens)/(wins + loses + evens) < 0.5: 
            loser_stat[int(yearth)].append(stock_name)
        else:
            even_stat[int(yearth)].append(stock_name)
           
        

计算留存转移概率
原理：
1. 找到上一年胜者组的股票
2. 剔除上一年胜者组中的数据，下一年未发生交易的股票
3. 与下一年败者组的股票比对，看是否有股票由胜者组进入了败者组


In [7]:
# winner_stat = {}         #{年度：stock}   代表stock胜率高于50%的股票，唯一，年度键值是int
# loser_stat = {}
# even_stat = {}
for yearth in sort(winner_stat.keys()):
    print yearth
    winner_current = set(winner_stat[yearth])         #当年的胜者列表
    
    total_nextyear = set(winner_stat[yearth+1] if winner_stat.has_key(yearth+1) else [])| \
                    set(loser_stat[yearth+1] if loser_stat.has_key(yearth+1) else [])| \
                    set(even_stat[yearth+1] if even_stat.has_key(yearth+1) else [])   #下一年的所有股票列表
    
    kickout_current = winner_current - total_nextyear      # 出现在当年胜者组，但是未在下一年出现的股票列表
    
    loser_nextyear = set(loser_stat[yearth+1]  if loser_stat.has_key(yearth+1) else [])            #下一年的败者组列表
    
    error_current = winner_current & loser_nextyear       #当年在胜者组，但是下一年进入败者组的股票列表
    
    print "错误率为: %f" % ( (len(error_current)/(len(winner_current) - len(kickout_current))) if (len(winner_current) - len(kickout_current)) != 0 else 0)

2005
错误率为: 0.250000
2006
错误率为: 0.190813
2007
错误率为: 0.274336
2008
错误率为: 0.264550
2009
错误率为: 0.220202
2010
错误率为: 0.215777
2011
错误率为: 0.050420
2012
错误率为: 0.134615
2013
错误率为: 0.202658
2014
错误率为: 0.138095
2015
错误率为: 0.112971
2016
错误率为: 0.110577
2017
错误率为: 0.043956
2018
错误率为: 0.000000


In [8]:

winner_stat[2016] = [1,2,3,8]
winner_stat[2017] = [2,3,4]
loser_stat[2017] = [6,7,8]
even_stat[2017] = [9,10,5]


w2016 =  set(winner_stat[2016])

print w2016


total2017 =  set(winner_stat[2017])|set(loser_stat[2017])|set(even_stat[2017])

print total2017


kickout2016 =  w2016 - total2017 


print kickout2016

denominator = len(w2016) - len(kickout2016)

print denominator


l2017 = set(loser_stat[2017])

print l2017

error2017 = w2016 & l2017
print error2017

numerator = len(error2017)

print numerator
# retA = [i for i in winner_stat['2015'] if i in winner_stat['2016']]
# # print len(winner_stat['2015'])
# # print len(retA)

# # # transfer_rate = len(winner_stat['2015'].intersection(winner_stat['2016'])) / len(winner_list['2015'])

# # print "在 %d 年，留存概率为: %f" % (2016,transfer_rate)
            
   

set([8, 1, 2, 3])
set([2, 3, 4, 5, 6, 7, 8, 9, 10])
set([1])
3
set([8, 6, 7])
set([8])
1
